In [1]:
from __future__ import absolute_import
from __future__ import print_function

import tensorflow as tf

In [10]:
def my_image_filter(input_images):
    conv1_weights=tf.Variable(tf.random_normal([5,5,32,32]),
                             name='conv1_weights')
    conv1_biases=tf.Variable(tf.zeros([32]),name='conv1_biases')
    conv1=tf.nn.conv2d(input_images,conv1_weights,strides=[1,1,1,1],padding='SAME')
    relu1=tf.nn.relu(conv1+conv1_biases)
    
    conv2_weights=tf.Variable(tf.random_normal([5,5,32,32]),
                             name='conv2_weights')
    conv2_biases=tf.Variable(tf.zeros([32]),name='conv2_biases')
    conv2=tf.nn.conv2d(relu1,conv2_weights,strides=[1,1,1,1],padding='SAME')
    return tf.nn.relu(conv2+conv2_biases)

In [11]:
image1=tf.Variable(tf.random_normal([1,28,28,32]))
image2=tf.Variable(tf.random_normal([1,28,28,32]))
result1=my_image_filter(image1)
result2=my_image_filter(image2)

# Sharing Variables

In [19]:
variables_dict={
    'conv1_weights':tf.Variable(tf.random_normal([5,5,32,32],name='conv1_weights')),
    'conv1_biases':tf.Variable(tf.zeros([32]),name='conv1_biases'),
    'conv2_weights':tf.Variable(tf.random_normal([5,5,32,32],name='conv2_weights')),
    'conv2_biases':tf.Variable(tf.zeros([32]),name='conv2_biases'),
}

def my_image_filter(input_images,variables_dict):
    conv1=tf.nn.conv2d(input_images,variables_dict['conv1_weights'],
                      strides=[1,1,1,1],padding='SAME')
    relu1=tf.nn.relu(conv1+variables_dict['conv1_biases'])
    
    conv2=tf.nn.conv2d(relu1,variables_dict['conv2_weights'],
                      strides=[1,1,1,1],padding='SAME')
    return tf.nn.relu(conv2+variables_dict['conv2_biases'])

In [20]:
result1=my_image_filter(image1,variables_dict)
result2=my_image_filter(image2,variables_dict)

# Variable Scope

In [22]:
def conv_relu(input,kernel_shape,bias_shape):
    # Create variable named 'weights'
    weights=tf.get_variable('weights',kernel_shape,initializer=tf.random_normal_initializer())
    # Create Variable named 'biases'
    biases=tf.get_variable('biases',bias_shape,initializer=tf.constant_initializer(0.0))
    conv=tf.nn.conv2d(input,weights,strides=[1,1,1,1],padding='SAME')
    return tf.nn.relu(conv+biases)

In [23]:
# tf.variable_scope() pushes a namespace for variables !
def my_image_filter(input_images):
    with tf.variable_scope('conv1'):
        # Variables created here will be named 'conv1/weights', 'conv1/biases'
        relu1=conv_relu(input_images,[5,5,32,32],[32])
    with tf.variable_scope('conv2'):
        # Variables created here will be named 'conv2/weights', 'conv2/biases'
        return conv_relu(relu1,[5,5,32,32],[32])

In [24]:
result1=my_image_filter(image1)
result2=my_image_filter(image2) # tf.get_variable() checks existing variables

ValueError: Variable conv1/weights already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-22-9c9cffccb199>", line 3, in conv_relu
    weights=tf.get_variable('weights',kernel_shape,initializer=tf.random_normal_initializer())
  File "<ipython-input-23-10b782c6ab87>", line 5, in my_image_filter
    relu1=conv_relu(input_images,[5,5,32,32],[32])
  File "<ipython-input-24-4a3ce1402666>", line 1, in <module>
    result1=my_image_filter(image1)


In [25]:
with tf.variable_scope('image_filters') as scope:
    result1=my_image_filter(image1)
    scope.reuse_variables()
    result2=my_image_filter(image2)

# Understanding tf.get_variable()

In [2]:
with tf.variable_scope('foo'): # tf.get_variable_scope().reuse==False
    v=tf.get_variable('v',[1])
assert v.name=='foo/v:0'

In [5]:
with tf.variable_scope('foo',reuse=True):
    v1=tf.get_variable('v',[1])
assert v1 is v

In [14]:
with tf.variable_scope('foo',reuse=True):
    v1=tf.get_variable('v',[1])
assert v1.name=='foo/v:0'

In [6]:
help(tf.get_variable)

Help on function get_variable in module tensorflow.python.ops.variable_scope:

get_variable(name, shape=None, dtype=None, initializer=None, regularizer=None, trainable=True, collections=None, caching_device=None, partitioner=None, validate_shape=True, custom_getter=None)
    Gets an existing variable with these parameters or create a new one.
    
    This function prefixes the name with the current variable scope
    and performs reuse checks. See the
    [Variable Scope How To](../../how_tos/variable_scope/index.md)
    for an extensive description of how reusing works. Here is a basic example:
    
    ```python
    with tf.variable_scope("foo"):
        v = tf.get_variable("v", [1])  # v.name == "foo/v:0"
        w = tf.get_variable("w", [1])  # w.name == "foo/w:0"
    with tf.variable_scope("foo", reuse=True)
        v1 = tf.get_variable("v")  # The same as v above.
    ```
    
    If initializer is `None` (the default), the default initializer passed in
    the variable scope wi

# Basics of tf.variable_scope()

In [2]:
with tf.variable_scope('foo'):
    with tf.variable_scope('bar'):
        v=tf.get_variable('v',[1])
assert v.name=='foo/bar/v:0'

In [3]:
with tf.variable_scope('foo1'):
    v=tf.get_variable('v',[1])
    tf.get_variable_scope().reuse_variables()
    v1=tf.get_variable('v',[1])
assert v1 is v

In [4]:
with tf.variable_scope('root'):
    # At start, the scope is not reusing
    assert tf.get_variable_scope().reuse==False
    with tf.variable_scope('foo'):
        # Opened a sub-scope, still not reusing
        assert tf.get_variable_scope().reuse==False
    with tf.variable_scope('foo',reuse=True):
        # Explicitly opened a reusing scope
        assert tf.get_variable_scope().reuse==True
        with tf.variable_scope('bar'):
            # Now sub-scope inherits the reuse flag
            assert tf.get_variable_scope().reuse==True
    # Exited the reusing scope, back to a non-reusing one
    assert tf.get_variable_scope().reuse==False

# Capturing variable scope

In [5]:
with tf.variable_scope('foo') as foo_scope:
    v=tf.get_variable('v',[1])
with tf.variable_scope(foo_scope):
    w=tf.get_variable('w',[1])
with tf.variable_scope(foo_scope,reuse=True):
    v1=tf.get_variable('v',[1])
    w1=tf.get_variable('w',[1])
    
assert v1 is v
assert w1 is w

In [6]:
with tf.variable_scope(foo_scope):
    assert foo_scope.name=='foo'
with tf.variable_scope('bar'):
    with tf.variable_scope('baz') as other_scope:
        assert other_scope.name=='bar/baz'
        with tf.variable_scope(foo_scope) as foo_scope2:
            assert foo_scope2.name=='foo' # Not changed

In [1]:
import tensorflow as tf

In [13]:
s=tf.InteractiveSession()
s.run(tf.initialize_all_variables())
with tf.variable_scope('foo',initializer=tf.constant_initializer(0.4)):
    tf.get_variable_scope().reuse_variables()
    v=tf.get_variable('v',[1])
    assert v.eval()==0.4
    w=tf.get_variable('w',[1],initializer=tf.constant_initializer(0.3))
    assert w.eval()==0.3
    with tf.variable_scope('bar'):
        v=tf.get_variable('v',[1])
        assert v.eval()==0.4
    with tf.variable_scope('baz',initializer=tf.constant_initializer(0.2)):
        v=tf.get_variable('v',[1])
        assert v.eval()==0.2

In [3]:
with tf.variable_scope('foo'):
    x=1.0+tf.get_variable('v',[1])
assert x.op.name=='foo/add'

In [2]:
with tf.variable_scope('foo1'):
    with tf.name_scope('bar'):
        v=tf.get_variable('v',[1])
        x=1.0+v
assert v.name=='foo1/v:0'
assert x.op.name=='foo1/bar/add'